**ref:** https://github.com/raul27868/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/SEMINARIO/Seminario(plantilla).ipynb

In [2]:
cifras = [x for x in range(1,10)]
signos = ['+','-','*','/']

## Espacio de posibilidades

- Cada operación en el espacio de posibilidades tiene 5 cifras (1-9) y 4 signos (+,-,*,/)
- Los números de no pueden repetirse
- Los signos no pueden repetirse
- Asumimos que la división solamente es viable si el resultado tiene resto 0 (división entera exacta)
- Para cifras contemplamos combinaciones sin repetición de 9 elementos elige 5: $\frac{n!}{(n-r)!}=\frac{9!}{(9-5)!}$
- Para signos contemplamos todas las permutaciones posibles de los 4 elementos sin repetición: $n! = 4!$
- **El espacio de posibilidades es:** $\frac{9!·4!}{4!}=9!$

## Resultado máximo (Analítico)

Encontrar el resultado máximo es un caso especial que puede razonarse fácilmente. Para tratar de obtener el resultado máximo podemos separar el problema en 2 segmentos distintos: primero un conjunto que tratamos de maximizar menos (segundo) un conjunto que queremos minimizar.

  1. Max_Resultado = Max_Conjunto - Min_Conjunto
 
- La operación crítica para obtener el resultado máximo es la división, por lo tanto, vamos a fijar que la división va a ser entre 1 para sacar ventaja a su elemento constante.
- Sin el 1, el conjunto mínimo es 2/1 (o 2).
- Para obtener el conjunto máximo partiremos de la multiplicación mayor (9*8) y añadiremos el siguiente valor mayor.

  2. Max_Resultado = 9*8+7-2/1 = 77
  
## Resultado mínimo (Analítico)

Análogamente, podemos utilizar un razonamiento parecido para obtener el resultado mínimo.

  3. Min_Resultado = A + Min_Conjunto - Max_Conjunto
  
Dónde A y Min_Conjunto deben ser lo menor posible y Max_Conjunto debe ser lo mayor posible.

- El Mayor Conjunto posible es 9*8
- Dado que no podemos repetir números, la división menor es 2 (4/2)
- A puede ser el número más pequeño del conjunto de cifras 1

  4. Min_Resultado = 1 + 4/2 - 9*8 = -69

## Fuerza Bruta

In [49]:
# Return all combinations of a collection using DFS strategy
def Combinations(collection, memory=[], depth=None):
    # If collection has one item, return the item in a list
    if len(collection) == 1:
        return [memory + collection]
    
    # If we have reached max depth, return a list 
    # of each item in a list
    if depth != None and depth <= 1:
        return [memory + [x] for x in collection]
    
    output = []
    for i,x in enumerate(collection):
        # Get all combinations of the sublist that doesn't include
        # the current item
        if depth == None:
            step = Combinations(collection[:i] + collection[i+1:], memory + [x])
        else:
            step = Combinations(collection[:i] + collection[i+1:], memory + [x], depth-1)
            
        # Prepend the current item to every combination
        #for y in step:
            #y.insert(0,x)
        output += step
    return output
    
        
def BruteForce(objective, numbers, signs, num_limit = 5, sign_limit = 4):
    sign_combinations = Combinations(signs, depth=sign_limit)
    num_combinations  = Combinations(numbers, depth=num_limit)
    
    for s in sign_combinations:
        for n in num_combinations:
            operation = "{}{}{}{}{}{}{}{}{}".format(n[0], s[0], n[1], s[1], n[2], s[2], n[3], s[3], n[4])

            if eval(operation) == objective:
                return operation
    return None


def BruteForceAll(numbers, signs, num_limit = 5, sign_limit = 4):
    sign_combinations = Combinations(signs, depth=sign_limit)
    num_combinations  = Combinations(numbers, depth=num_limit)
    operations = {}
    
    for s in sign_combinations:
        for n in num_combinations:
            operation = "{}{}{}{}{}{}{}{}{}".format(n[0], s[0], n[1], s[1], n[2], s[2], n[3], s[3], n[4])
            value = eval(operation)
            
            if value%1 != 0:
                continue
            
            if value in operations.keys():
                continue
            operations[value] = operation
    return operations

In [46]:
BruteForce(78, cifras, signos)

In [51]:
operations = BruteForceAll(cifras, signos)

In [59]:
min_value = int(min(operations.keys()))
max_value = int(max(operations.keys()))

print("Minimum result:", min_value, "=", operations[min_value])
print("Maximum result: ", max_value, "=", operations[max_value])
print("Contains all integers in-between:", all([x in operations.keys() for x in range(min_value, max_value+1)]))

Minimum result: -69 = 1+4/2-8*9
Maximum result:  77 = 7+8*9-2/1
Contains all integers in-between: True


El algoritmo de Fuerza Bruta se compone de dos recorridos DFS con orden $O(n!)$ y un doble recorrido para generar todas las combinaciones con orden $O(n^2)$. Así, el algoritmo de Fuerza Bruta es $O(n!)$

## Refinar

Como hemos visto en el razonamiento analítico del máximo y el mínimo, este problema nos presenta con, básicamente, una operación con 3 términos:

5. {Resultado = A - D + M} ó {Resultado = A + D - M} 

En este caso, podemos considerar A como el sesgo inicial, D es el término de la división y M es el término de la multiplicación.

Así, el problema se reduce a encontrar una combinación de 3 términos.

In [60]:
def Refinado(n, numbers, signs):
    # nos creamos un dict con todas las multiplicaciones posibles
    # y otro dict con todas las divisiones posibles
    
    Mul = {}
    Div = {}
    
    for i,x in enumerate(numbers):
        for y in numbers[i+1:]:
            if x*y in Mul.keys():
                Mul[x*y].append( (x,y) )
            else:
                Mul[x*y] = [(x,y)]
                
            if x%y == 0 and x/y in Div.keys():
                Div[x//y].append( (x,y) )
            else:
                Div[x//y] = [(x,y)]
                
    # Poda:
    # Partiendo de la propiedad commutativa de la suma,
    # fijamos la estructura a A +/- D -/+ M.
    # Es decir, generaremos el árbol por orden: sesgo, división, multiplicación
    
    # Usaremos tuplas con el formato (cota_inferior, cota_superior, conjunto_numeros)
    explore = [(min(Div.keys()) - max(Mul.keys()), 9 - min(Div.keys()) + max(Mul.keys()), [])]
    print(explore)
Refinado(0, cifras, signos)

[(-72, 81, [])]
